In [1]:
%run common_definitions.py
%run plotting_functions.py

In [2]:
import brightway2 as bw
from premise_gwp import add_premise_gwp

import pandas as pd
import xarray as xr
import os
import yaml

In [3]:
bw.projects.set_current("GettingStarted_EI_3.10")

In [4]:
add_premise_gwp()

RELICS requires the name of your biosphere database.
Please enter the name of your biosphere database as it appears in your project.
Databases dictionary with 2 object(s):
	ecoinvent-3.10-biosphere
	ecoinvent-3.10-cutoff
Adding ('IPCC 2021', 'climate change', 'GWP 20a, incl. H and bio CO2')
Converting to ei 3.10 biosphere names
Applying strategy: csv_restore_tuples
Applying strategy: csv_numerize
Applying strategy: csv_drop_unknown
Applying strategy: set_biosphere_type
Applying strategy: drop_unspecified_subcategories
Applying strategy: link_iterable_by_fields
Applying strategy: drop_falsey_uncertainty_fields_but_keep_zeros
Applying strategy: convert_uncertainty_types_to_integers
Applied 8 strategies in 0.08 seconds
Applying strategy: drop_unlinked_cfs
Applied 1 strategies in 0.00 seconds
0 unlinked flows:
Wrote 1 LCIA methods with 208 characterization factors
Adding ('IPCC 2021', 'climate change', 'GWP 100a, incl. H and bio CO2')
Converting to ei 3.10 biosphere names
Applying strategy

## Split climate methods

In [5]:
base_methods = [
    ('IPCC 2021', 'climate change', 'GWP 100a, incl. H and bio CO2'),
    ('IPCC 2021', 'climate change', 'GWP 20a, incl. H and bio CO2')
]

In [6]:
for method in base_methods:
    M = bw.Method(method)
    unit = ""
    if "unit" in M.metadata:
        unit = M.metadata["unit"]

    CFs_bio = []
    CFs_landuse = []
    CFs_fossil = []
    for flow in bw.Method(method).load():
        act = bw.get_activity(flow[0])
        if "non-fossil" in act["name"] or "in air" in act["name"]:
            CFs_bio.append(list(flow))
        elif "soil or biomass" in act["name"]:
            CFs_landuse.append(list(flow))
        else:
            CFs_fossil.append(list(flow))

    new_key = (method[0], method[1]+": biogenic", method[2].split(",")[0])    
    new_method = bw.Method(new_key)
    new_method.validate(CFs_bio)
    new_method.register(unit=unit)
    new_method.write(CFs_bio)

    new_key = (method[0], method[1]+": land use", method[2].split(",")[0])    
    new_method = bw.Method(new_key)
    new_method.validate(CFs_landuse)
    new_method.register(unit=unit)
    new_method.write(CFs_landuse)

    new_key = (method[0], method[1]+": fossil", method[2].split(",")[0])    
    new_method = bw.Method(new_key)
    new_method.validate(CFs_fossil)
    new_method.register(unit=unit)
    new_method.write(CFs_fossil)

## Add Eco-costs metals

In [7]:
table = pd.read_csv("../data/cfs_ecocosts_metals.csv")
table.dropna(subset="characterization factor", inplace=True)

In [8]:
cfs_costs = pd.read_csv("../data/cfs_cost_methods.csv")

In [9]:
db = bw.Database("ecoinvent-3.10-biosphere")
resources = [flow for flow in db if flow["categories"] == ("natural resource", "in ground")]

new_key = ("Eco-costs", "Material scarcity", "VAR(95)")
CFs = []
for idx, row in table.iterrows():
    matches = [flow for flow in resources if flow["name"] == row["name"] and str(flow["categories"]) == row["categories"]]
    if len(matches) == 1:
        CFs.append([matches[0].key, row["characterization factor"]])
    else:
        print("Error for {}: {} match(es). Flow dropped.".format(row["name"], len(matches)))

new_method = bw.Method(new_key)
new_method.validate(CFs)
new_method.register(unit="EUR2023")
new_method.write(CFs)

Error for Aluminium, 24% in bauxite, 11% in crude ore: 0 match(es). Flow dropped.
Error for Cadmium, 0.30% in sulfide, Cd 0.18%, Pb, Zn, Ag, In: 0 match(es). Flow dropped.
Error for Chromium, 25.5% in chromite, 11.6% in crude ore: 0 match(es). Flow dropped.
Error for Cobalt, Co 5.0E-2%, in mixed ore: 0 match(es). Flow dropped.
Error for Copper, 1.42% in sulfide, Cu 0.81% and Mo 8.2E-3% in crude ore: 0 match(es). Flow dropped.
Error for Copper, Cu 0.38%, in mixed ore: 0 match(es). Flow dropped.
Error for Copper, 0.99% in sulfide, Cu 0.36% and Mo 8.2E-3% in crude ore: 0 match(es). Flow dropped.
Error for Copper, Cu 6.8E-1%, in mixed ore: 0 match(es). Flow dropped.
Error for Copper, 0.52% in sulfide, Cu 0.27% and Mo 8.2E-3% in crude ore: 0 match(es). Flow dropped.
Error for Copper, Cu 0.2%, in mixed ore: 0 match(es). Flow dropped.
Error for Copper, 1.18% in sulfide, Cu 0.39% and Mo 8.2E-3% in crude ore: 0 match(es). Flow dropped.
Error for Copper, 1.25% in sulfide, Cu 0.24% and Zn 0,1% in

In [77]:
def add_methods_from_dataframe(df):
    methods2add = list(df["method"].unique())

    for m in methods2add:
        sel = df[df["method"] == m].copy()
        new_key = tuple(list(m.split(", ")))
        unit = sel["unit"].iloc[0]
        keys = list(sel["mapped key"])
        cfs = list(sel["CF"])
        new_cfs = [[eval(k), cf] for k, cf in zip(keys, cfs)]
        new_method = bw.Method(new_key)
        new_method.validate(new_cfs)
        new_method.register(unit=unit)
        new_method.write(new_cfs)

        print("Added method {}".format(str(new_method)))

## Add old ReCiPe methods

In [58]:
df_recipe = pd.read_csv("../data/cfs_old_recipe_methods_mapped.csv")
name_mapping = yaml.safe_load(open("../data/mapping_ei310.yaml", "r"))

def map_flow_name(x):
    if x in name_mapping.keys():
        return name_mapping[x]
    else:
        return x

In [62]:
df_recipe["new flow name"] = df_recipe["new flow name"].apply(map_flow_name)

In [63]:
df_recipe[df_recipe["new flow name"].isna()]

,method,unit,new flow name,flow categories,CF


In [64]:
df_recipe.dropna(subset="new flow name", inplace=True)

Remove exchanges into "('air', 'lower stratosphere + upper troposphere')"

In [65]:
db = bw.Database("ecoinvent-3.10-biosphere")

not_change = [act["name"] for act in db if act["categories"] == ('air', 'lower stratosphere + upper troposphere')]
not_change

['Particulate Matter, < 2.5 um',
 'NMVOC, non-methane volatile organic compounds',
 'Carbon dioxide, fossil',
 'Chromium III',
 'Nitrogen oxides',
 'Carbon monoxide, fossil',
 'Selenium IV',
 'Water',
 'Copper ion',
 'Sulfur dioxide',
 'Nickel II',
 'Zinc II',
 'Cadmium II',
 'Mercury II',
 'Lead II']

In [66]:
idx_remove = df_recipe[df_recipe["flow categories"] == "('air', 'lower stratosphere + upper troposphere')"][~df_recipe["new flow name"].isin(not_change)].index

C:\Users\davidba\AppData\Local\Temp\ipykernel_15924\1340859795.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  idx_remove = df_recipe[df_recipe["flow categories"] == "('air', 'lower stratosphere + upper troposphere')"][~df_recipe["new flow name"].isin(not_change)].index


In [67]:
df_recipe = df_recipe.drop(index=idx_remove)

### Checking names

In [68]:
db = bw.Database("ecoinvent-3.10-biosphere")

all_names = list(set([act["name"] for act in db]))

needed_names = list(df_recipe["new flow name"].unique())

for n in needed_names:
    if n not in all_names:
        print("{} not found.".format(n))


In [69]:
db = bw.Database("ecoinvent-3.10-biosphere")

flowlist = []
categorieslist = []
keylist = []

needed_flows = list(df_recipe.groupby(["new flow name", "flow categories"]).size().index)
flow_mapping = {(act["name"], str(act["categories"])): act.key for act in db}

for f in needed_flows:
    if f not in flow_mapping.keys():
        print(f[0])


In [18]:
df_recipe["mapped key"] = [flow_mapping[i] for i in df_recipe.set_index(["new flow name", "flow categories"]).index]
df_recipe[["method", "unit", "new flow name", "flow categories", "mapped key", "CF"]].to_csv("../data/cfs_old_recipe_methods_mapped_w_keys_ei310.csv", index=False)

### Add methods to brightway

In [19]:
recipe_cfs = pd.read_csv("../data/cfs_old_recipe_methods_mapped_w_keys_ei310.csv").dropna(subset="mapped key")

add_methods_from_dataframe(recipe_cfs)

Added method Brightway2 Method: ReCiPe Endpoint (E,A) w/o LT: ecosystem quality w/o LT: freshwater ecotoxicity w/o LT
Added method Brightway2 Method: ReCiPe Endpoint (E,A) w/o LT: ecosystem quality w/o LT: freshwater eutrophication w/o LT
Added method Brightway2 Method: ReCiPe Endpoint (E,A) w/o LT: ecosystem quality w/o LT: urban land occupation w/o LT
Added method Brightway2 Method: ReCiPe Endpoint (E,A) w/o LT: ecosystem quality w/o LT: climate change: ecosystems w/o LT
Added method Brightway2 Method: ReCiPe Endpoint (E,A) w/o LT: ecosystem quality w/o LT: terrestrial ecotoxicity w/o LT
Added method Brightway2 Method: ReCiPe Endpoint (E,A) w/o LT: ecosystem quality w/o LT: agricultural land occupation w/o LT
Added method Brightway2 Method: ReCiPe Endpoint (E,A) w/o LT: ecosystem quality w/o LT: marine ecotoxicity w/o LT
Added method Brightway2 Method: ReCiPe Endpoint (E,A) w/o LT: ecosystem quality w/o LT: natural land transformation w/o LT
Added method Brightway2 Method: ReCiPe End

## EDIP methods

In [90]:
df_edip= pd.read_csv("../data/cfs_edip_methods.csv")

In [91]:
df_edip["new flow name"] = df_edip["flow name"].apply(map_flow_name)

In [92]:
df_edip[df_edip["new flow name"].isna()]

,method,unit,flow name,flow categories,CF,new flow name


In [72]:
df_edip.dropna(subset="new flow name", inplace=True)

In [73]:
db = bw.Database("ecoinvent-3.10-biosphere")

not_change = [act["name"] for act in db if act["categories"] == ('air', 'lower stratosphere + upper troposphere')]
idx_remove = df_edip[df_edip["flow categories"] == "('air', 'lower stratosphere + upper troposphere')"][~df_edip["new flow name"].isin(not_change)].index
df_edip = df_edip.drop(index=idx_remove)

C:\Users\davidba\AppData\Local\Temp\ipykernel_15924\2530644479.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  idx_remove = df_edip[df_edip["flow categories"] == "('air', 'lower stratosphere + upper troposphere')"][~df_edip["new flow name"].isin(not_change)].index


In [74]:
db = bw.Database("ecoinvent-3.10-biosphere")

all_names = list(set([act["name"] for act in db]))

needed_names = list(df_edip["new flow name"].unique())

for n in needed_names:
    if n not in all_names:
        print("{} not found.".format(n))

In [75]:
db = bw.Database("ecoinvent-3.10-biosphere")

flowlist = []
categorieslist = []
keylist = []

needed_flows = list(df_edip.groupby(["new flow name", "flow categories"]).size().index)
flow_mapping = {(act["name"], str(act["categories"])): act.key for act in db}

for f in needed_flows:
    if f not in flow_mapping.keys():
        print(f[1])

In [76]:
df_edip["mapped key"] = [flow_mapping[i] for i in df_edip.set_index(["new flow name", "flow categories"]).index]
df_edip[["method", "unit", "new flow name", "flow categories", "mapped key", "CF"]].to_csv("../data/cfs_edip_methods_mapped_w_keys_ei310.csv", index=False)

In [78]:
edip_cfs = pd.read_csv("../data/cfs_edip_methods_mapped_w_keys_ei310.csv").dropna(subset="mapped key")

add_methods_from_dataframe(edip_cfs)

Added method Brightway2 Method: EDIP 2003: acidification: acidification
Added method Brightway2 Method: EDIP 2003: ecotoxicity: acute: in water
Added method Brightway2 Method: EDIP 2003: ecotoxicity: chronic: in soil
Added method Brightway2 Method: EDIP 2003: ecotoxicity: chronic: in water
Added method Brightway2 Method: EDIP 2003: ecotoxicity: in sewage treatment plants
Added method Brightway2 Method: EDIP 2003: eutrophication: combined potential
Added method Brightway2 Method: EDIP 2003: eutrophication: separate N potential
Added method Brightway2 Method: EDIP 2003: eutrophication: separate P potential
Added method Brightway2 Method: EDIP 2003: eutrophication: terrestrial eutrophication
Added method Brightway2 Method: EDIP 2003: global warming: GWP 100a
Added method Brightway2 Method: EDIP 2003: global warming: GWP 20a
Added method Brightway2 Method: EDIP 2003: global warming: GWP 500a
Added method Brightway2 Method: EDIP 2003: human toxicity: via air
Added method Brightway2 Method: 

## IMPACT 2002+

In [86]:
df_impactplus= pd.read_csv("../data/cfs_impactplus_methods.csv")

In [87]:
df_impactplus["new flow name"] = df_impactplus["flow name"].apply(map_flow_name)

In [88]:
df_impactplus[df_impactplus["new flow name"].isna()]

,method,unit,flow name,flow categories,CF,new flow name


In [93]:
df_impactplus.dropna(subset="new flow name", inplace=True)

In [94]:
db = bw.Database("ecoinvent-3.10-biosphere")

not_change = [act["name"] for act in db if act["categories"] == ('air', 'lower stratosphere + upper troposphere')]
idx_remove = df_impactplus[df_impactplus["flow categories"] == "('air', 'lower stratosphere + upper troposphere')"][~df_impactplus["new flow name"].isin(not_change)].index
df_impactplus = df_impactplus.drop(index=idx_remove)

C:\Users\davidba\AppData\Local\Temp\ipykernel_15924\891340877.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  idx_remove = df_impactplus[df_impactplus["flow categories"] == "('air', 'lower stratosphere + upper troposphere')"][~df_impactplus["new flow name"].isin(not_change)].index


In [95]:
db = bw.Database("ecoinvent-3.10-biosphere")

all_names = list(set([act["name"] for act in db]))

needed_names = list(df_impactplus["new flow name"].unique())

for n in needed_names:
    if n not in all_names:
        print("{} not found.".format(n))

In [96]:
db = bw.Database("ecoinvent-3.10-biosphere")

flowlist = []
categorieslist = []
keylist = []

needed_flows = list(df_impactplus.groupby(["new flow name", "flow categories"]).size().index)
flow_mapping = {(act["name"], str(act["categories"])): act.key for act in db}

for f in needed_flows:
    if f not in flow_mapping.keys():
        print(f[1])

In [97]:
df_impactplus["mapped key"] = [flow_mapping[i] for i in df_impactplus.set_index(["new flow name", "flow categories"]).index]
df_impactplus[["method", "unit", "new flow name", "flow categories", "mapped key", "CF"]].to_csv("../data/cfs_impactplus_methods_mapped_w_keys_ei310.csv", index=False)

In [98]:
impactplus_cfs = pd.read_csv("../data/cfs_impactplus_methods_mapped_w_keys_ei310.csv").dropna(subset="mapped key")

add_methods_from_dataframe(impactplus_cfs)

Added method Brightway2 Method: IMPACT 2002+ (Endpoint): climate change: climate change
Added method Brightway2 Method: IMPACT 2002+ (Endpoint): climate change: total
Added method Brightway2 Method: IMPACT 2002+ (Endpoint): ecosystem quality: aquatic ecotoxicity
Added method Brightway2 Method: IMPACT 2002+ (Endpoint): ecosystem quality: land occupation
Added method Brightway2 Method: IMPACT 2002+ (Endpoint): ecosystem quality: terrestrial acidification & nutrification
Added method Brightway2 Method: IMPACT 2002+ (Endpoint): ecosystem quality: terrestrial ecotoxicity
Added method Brightway2 Method: IMPACT 2002+ (Endpoint): ecosystem quality: total
Added method Brightway2 Method: IMPACT 2002+ (Endpoint): human health: human toxicity
Added method Brightway2 Method: IMPACT 2002+ (Endpoint): human health: ionising radiation
Added method Brightway2 Method: IMPACT 2002+ (Endpoint): human health: ozone layer depletion
Added method Brightway2 Method: IMPACT 2002+ (Endpoint): human health: photo

## Monetization factors

In [99]:
df2 = pd.read_csv("../data/mfs_all_factors.csv").dropna(subset="LCIA method")

In [100]:
needed_methods = list(df2["LCIA method"].unique())
methods = [m for m in list(bw.methods) if ", ".join(list(m)) in needed_methods]

In [101]:
cc_methods = [
    ("IPCC 2021", "climate change: biogenic", "GWP 100a"),
    ("IPCC 2021", "climate change: land use", "GWP 100a"),
    ("IPCC 2021", "climate change: fossil", "GWP 100a")
]
methods = methods + cc_methods

In [102]:
print("{} methods needed, {} methods found".format(len(needed_methods)+len(cc_methods), len(methods)))

73 methods needed, 73 methods found


In [103]:
all_methods = [", ".join(list(m)) for m in bw.methods]

In [104]:
for m in needed_methods:
    if not m in all_methods:
        print(m)

## Create mean costs brightway method

In [105]:
mfs_mc_sample = xr.open_dataarray("../data/mfs_monte_carlo_sample_euro{}_larger.nc".format(str(EURO_REF_YEAR)))

In [106]:
mean_mfs_by_category = mfs_mc_sample.mean(dim="sample index")

In [107]:
def sum_up_CFs(CFlist):
    keys = [CF[0] for CF in CFlist]
    values = [CF[1] for CF in CFlist]
    df = pd.DataFrame({"keys": keys, "values": values})
    summed = df.groupby("keys").sum()
    return [[k, v] for k, v in zip(summed.index, summed["values"])]

In [108]:
for ic in list(mean_mfs_by_category.coords["impact category"].values):
    MFs = mean_mfs_by_category.sel({"impact category": ic})
    new_key = ("Monetization perspectives", "Monte Carlo mean", "by impact category", ic)
    nonzero = MFs.loc[MFs != 0]
    relevant_methods = [m for m in bw.methods if ", ".join(list(m)) in list(nonzero.coords["LCIA method"].values)]
    CFs = []
    for m in relevant_methods:
        MF = float(MFs.sel({"LCIA method": ", ".join(list(m))}))
        for flow in bw.Method(m).load():
            new_CF = list(flow)
            new_CF[1] = new_CF[1] * MF
            CFs.append(new_CF)
    CFs = sum_up_CFs(CFs)
    new_method = bw.Method(new_key)
    new_method.validate(CFs)
    new_method.register(unit="EUR{}".format(EURO_REF_YEAR))
    new_method.write(CFs)     

In [109]:
ics_to_add = [k for k in impact2color.keys() if "fossil" not in k]

In [110]:
mean_mfs_total = mean_mfs_by_category.sel({"impact category": ics_to_add}).sum(dim="impact category")

In [111]:
new_key = ("Monetization perspectives", "Monte Carlo mean", "total costs")
CFs = []
for mstr in list(mean_mfs_total.coords["LCIA method"].values):
    mtuple = [m for m in bw.methods if ", ".join(list(m)) == mstr][0]
    MF = float(mean_mfs_total.sel({"LCIA method": mstr}))
    for flow in bw.Method(mtuple).load():
        new_CF = list(flow)
        new_CF[1] = new_CF[1] * MF
        CFs.append(new_CF)
CFs = sum_up_CFs(CFs)
new_method = bw.Method(new_key)
new_method.validate(CFs)
new_method.register(unit="EUR{}".format(EURO_REF_YEAR))
new_method.write(CFs)

In [112]:
mfs_perspectives_cc = xr.open_dataarray("../data/mfs_perspectives_extended_euro_{}.nc".format(EURO_REF_YEAR))

In [113]:
mfs_perspectives = mfs_perspectives_cc.sel({"impact category": [k for k in impact2color.keys()]})

In [114]:
for perspective in list(mfs_perspectives.coords["perspective"].values):
    for ic in list(mfs_perspectives.coords["impact category"].values):
        MFs = mfs_perspectives.sel({"impact category": ic, "perspective": perspective})
        new_key = ("Monetization perspectives", perspective, "by impact category", ic)
        nonzero = MFs.loc[MFs != 0]
        if nonzero.size == 0:
            print("{} no covered in {}".format(ic, perspective))
        else:
            relevant_methods = [m for m in bw.methods if ", ".join(list(m)) in list(nonzero.coords["LCIA method"].values)]
            CFs = []
            for m in relevant_methods:
                MF = float(MFs.sel({"LCIA method": ", ".join(list(m))}))
                for flow in bw.Method(m).load():
                    new_CF = list(flow)
                    new_CF[1] = new_CF[1] * MF
                    CFs.append(new_CF)
            print("{} CFs for {}".format(len(CFs), str(new_key)))
            CFs = sum_up_CFs(CFs)
            new_method = bw.Method(new_key)
            new_method.validate(CFs)
            new_method.register(unit="EUR{}".format(EURO_REF_YEAR))
            new_method.write(CFs) 

33 CFs for ('Monetization perspectives', 'budget constraint', 'by impact category', 'acidification')
208 CFs for ('Monetization perspectives', 'budget constraint', 'by impact category', 'climate change')
1286 CFs for ('Monetization perspectives', 'budget constraint', 'by impact category', 'ecotoxicity')
122 CFs for ('Monetization perspectives', 'budget constraint', 'by impact category', 'eutrophication')
fossil resources no covered in budget constraint
607 CFs for ('Monetization perspectives', 'budget constraint', 'by impact category', 'human toxicity')
130 CFs for ('Monetization perspectives', 'budget constraint', 'by impact category', 'ionizing radiation')
30 CFs for ('Monetization perspectives', 'budget constraint', 'by impact category', 'land use')
43 CFs for ('Monetization perspectives', 'budget constraint', 'by impact category', 'metal/mineral resources')
76 CFs for ('Monetization perspectives', 'budget constraint', 'by impact category', 'ozone depletion')
28 CFs for ('Monetizati

In [115]:
ics_to_add = [k for k in impact2color.keys() if "fossil" not in k]

mfs_perspectives_total = mfs_perspectives.sel({"impact category": ics_to_add}).sum(dim="impact category")

In [116]:
for perspective in list(mfs_perspectives_total.coords["perspective"].values):
    new_key = ("Monetization perspectives", perspective, "total costs")
    CFs = []
    MFs = mfs_perspectives_total.sel(perspective=perspective)
    nonzero = MFs.loc[MFs != 0]
    relevant_methods = [m for m in bw.methods if ", ".join(list(m)) in list(nonzero.coords["LCIA method"].values)]
    for m in relevant_methods:
        mtuple = [m for m in bw.methods if ", ".join(list(m)) == mstr][0]
        MF = float(MFs.sel({"LCIA method": ", ".join(list(m))}))
        for flow in bw.Method(m).load():
            new_CF = list(flow)
            new_CF[1] = new_CF[1] * MF
            CFs.append(new_CF)
    CFs = sum_up_CFs(CFs)
    new_method = bw.Method(new_key)
    new_method.validate(CFs)
    new_method.register(unit="EUR{}".format(EURO_REF_YEAR))
    new_method.write(CFs)

## Export methods as csv

In [117]:
def get_method_dataframe(m):
    M = bw.Method(m)
    CFs = []
    names = []
    categories = []
    for flow in M.load():
        CFs.append(flow[1])
        act = bw.get_activity(flow[0])
        names.append(act["name"])
        categories.append(act["categories"])

    df = pd.DataFrame(
        {
            "flow name": names,
            "flow categories": categories,
            "CF": CFs
        },
    )

    df["method"] = ", ".join(list(m))
    df["unit"] = M._get_metadata()["unit"]

    return df

In [118]:
dflist = []

for m in list(bw.methods):
    if "Monetization perspectives" in m[0]:
        dflist.append(get_method_dataframe(m))

pd.concat(dflist)[["method", "unit", "flow name", "flow categories", "CF"]].to_csv("../data/cfs_cost_methods_ei310.csv", index=False)


In [119]:
df_cost_methods = pd.read_csv("../data/cfs_cost_methods_ei310.csv")
df_cost_methods

,method,unit,flow name,flow categories,CF
0,"Monetization perspectives, Monte Carlo mean, b...",EUR2022,"1,1-Dichloro-1-fluoroethane","('air', 'urban air close to ground')",112.299643
1,"Monetization perspectives, Monte Carlo mean, b...",EUR2022,Ethylene dichloride,"('air',)",0.169755
2,"Monetization perspectives, Monte Carlo mean, b...",EUR2022,Dichlorofluoromethane,"('air',)",20.892957
3,"Monetization perspectives, Monte Carlo mean, b...",EUR2022,"2,2-Dichloro-1,1,1-trifluoroethane","('air',)",11.804521
4,"Monetization perspectives, Monte Carlo mean, b...",EUR2022,"Methane, bromochlorodifluoro-, Halon 1211","('air', 'low population density, long-term')",252.021292
...,...,...,...,...,...
32410,"Monetization perspectives, taxation costs, tot...",EUR2022,Propanol,"('air',)",22.360343
32411,"Monetization perspectives, taxation costs, tot...",EUR2022,Fluometuron,"('soil', 'agricultural')",51.510475
32412,"Monetization perspectives, taxation costs, tot...",EUR2022,Ethylene glycol monoethyl ether,"('air', 'urban air close to ground')",15.509400
32413,"Monetization perspectives, taxation costs, tot...",EUR2022,Beta-cyfluthrin,"('air', 'non-urban air or from high stacks')",1498.586461


In [120]:
db = bw.Database("ecoinvent-3.10-biosphere")

needed_flows = list(df_cost_methods.groupby(["flow name", "flow categories"]).size().index)
flow_mapping = {(act["name"], str(act["categories"])): act.key for act in db}

for f in needed_flows:
    if f not in flow_mapping.keys():
        print(f[0], f[1])

In [121]:
for mstr in needed_methods:
    matches = [m for m in bw.methods if ", ".join(list(m)) == mstr]
    if len(matches) == 1:
        M = bw.Method(matches[0])

        counter = 0
        for flow in M.load():
            if flow[0][0] != "ecoinvent-3.10-biosphere":
                counter += 1
        
        if counter > 0:
            print("{}: {} not in correct biosphere.".format(matches[0], counter))
    else:
        print("{}: Not found!".format(mstr))

In [122]:
# for m in list(bw.methods):
#     if "Monetization perspectives" in m[0]:
#         print("Removing "+str(m))
#         bw.Method(m).deregister()

Removing ('Monetization perspectives', 'Monte Carlo mean', 'by impact category', 'climate change')
Removing ('Monetization perspectives', 'Monte Carlo mean', 'by impact category', 'acidification')
Removing ('Monetization perspectives', 'Monte Carlo mean', 'by impact category', 'particulate matter formation')
Removing ('Monetization perspectives', 'Monte Carlo mean', 'by impact category', 'ozone depletion')
Removing ('Monetization perspectives', 'Monte Carlo mean', 'by impact category', 'ionizing radiation')
Removing ('Monetization perspectives', 'Monte Carlo mean', 'by impact category', 'fossil resources')
Removing ('Monetization perspectives', 'Monte Carlo mean', 'by impact category', 'metal/mineral resources')
Removing ('Monetization perspectives', 'Monte Carlo mean', 'by impact category', 'water use')
Removing ('Monetization perspectives', 'Monte Carlo mean', 'by impact category', 'human toxicity')
Removing ('Monetization perspectives', 'Monte Carlo mean', 'by impact category', 'eut